In [1]:
import re
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [2]:
df = pd.read_csv("GOOGL_day.csv")
df = df.iloc[::-1]
del df['Unnamed: 0']
df

,volume,open,close,high,low,date
1257,1663302.0,930.985,940.810,944.660,929.7900,2017-07-07 04:00:00
1256,1451460.0,941.950,951.000,953.130,941.9500,2017-07-10 04:00:00
1255,1461247.0,950.520,953.530,954.890,945.1200,2017-07-11 04:00:00
1254,1602115.0,960.860,967.660,969.630,957.0400,2017-07-12 04:00:00
1253,1524571.0,970.800,968.850,978.700,964.8000,2017-07-13 04:00:00
...,...,...,...,...,...,...
4,1235841.0,2231.040,2234.030,2263.205,2217.4501,2022-06-29 04:00:00
3,2158453.0,2200.000,2179.260,2217.760,2135.0000,2022-06-30 04:00:00
2,1773783.0,2158.660,2174.750,2185.000,2134.6100,2022-07-01 04:00:00
1,1970154.0,2142.010,2265.260,2267.535,2114.6300,2022-07-05 04:00:00


In [3]:
google_stock = df['close'].values.astype('float32')
google_stock = google_stock.reshape(len(google_stock), 1)
google_stock

array([[ 940.81 ],
       [ 951.   ],
       [ 953.53 ],
       ...,
       [2174.75 ],
       [2265.26 ],
       [2297.594]], dtype=float32)

In [4]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
google_stock_n = scaler.fit_transform(google_stock)
google_stock_n

array([[0.00960547],
       [0.01451421],
       [0.01573294],
       ...,
       [0.6040175 ],
       [0.64761794],
       [0.6631938 ]], dtype=float32)

In [5]:
def train_test_split(stock):
  train_size = int(len(stock) * 0.80)
  test_size = len(stock) - train_size
  train, test = stock[0:train_size,:], stock[train_size:len(stock),:]
  return train, test

In [6]:
def create_dataset(dataset, look_back=1):
	dataX, dataY = [], []
	for i in range(len(dataset)-look_back-1):
		a = dataset[i:(i+look_back), 0]
		dataX.append(a)
		dataY.append(dataset[i + look_back, 0])
	return np.array(dataX), np.array(dataY)
def create_trainTest():
	train, test = train_test_split(google_stock_n)
	# reshape into X=t and Y=t+1
	look_back = 1
	trainX, trainY = create_dataset(train, look_back)
	testX, testY = create_dataset(test, look_back)
	trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
	testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
	return trainX, testX, trainY, testY
trainX, testX, trainY, testY = create_trainTest()


In [9]:
trainX, testX, trainY, testY = create_trainTest()
model = Sequential()
# model.add(tf.keras.Input(shape=(10,8)))
# model.add(tf.keras.layers.Conv1D(32,3,input_shape = (10, 8), activation = 'relu'))
# model.add(tf.keras.layers.MaxPool1D())
model.add(LSTM(units=50,return_sequences=True,input_shape=(1, 32), activation = 'relu'))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(trainX, trainY,epochs=1,batch_size=32)



ValueError: in user code:

    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/engine/training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/utils/traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb)